In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import os
path="/content/drive/My Drive/Stat_Gene/Data"

os.chdir(path)
os.listdir(path)

import time
import math
import numpy as np
import pandas as pd
from csv import reader
from scipy import special
from scipy.optimize import minimize
from scipy.optimize import Bounds

import matplotlib.pyplot as plt
from scipy.stats import chi2

In [3]:
import cupy as cp

# Version 5 for small dataset

In [4]:
# E - step self-defined kernel
sigmoid2_kernel = cp.ElementwiseKernel(
    'float32 x',
    'float32 mu',
    '''
    mu = 1 / (1 + exp(- x ));
    ''',
    'sigmoid2_kernel'
)

diff_trigamma_kernel = cp.ElementwiseKernel(
    'float32 x_small, int32 diff',  #for example  x_big = u/phi + y, x_small = u/phi
    'float32 result',
    '''
    result = 0 ;
    for (int i = 0; i < diff; i++){
      result = result - 1 / ( ( i + x_small) * (i + x_small) );
    }
    ''',
    'diff_trigamma_kernel'
)

diff_digamma_kernel = cp.ElementwiseKernel(
    'float32 x_small, int32 diff',  #for example  x_big = u/phi + y, x_small = u/phi
    'float32 result',
    '''
    result = 0 ;
    for (int i = 0; i < diff; i++){
      result = result + 1 / (  i + x_small ) ;
    }
    ''',
    'diff_digamma_kernel'
)



# M-step Define Kernel Function
dot_product_kernel = cp.ReductionKernel(
    'T x, T y',            # 输入变量的类型和名字
    'T z',                 # 输出变量的类型和名字
    'x * y',               # 输入元素的映射表达式（元素级乘法）
    'a + b',               # 归约操作（加法）
    'z = a',               # 输出操作（直接赋值）
    '0',                   # 归约的初始值
    'dot_product'          # 核函数的名称
)


sigmoid_kernel = cp.ElementwiseKernel(
    'float32 lp,float32 rand,float32 pre, float32 zmat',
    'float32 mu',
    '''
    mu = 1 / (1 + exp(-(lp + rand + zmat / pre)));
    ''',
    'sigmoid_kernel'
)

beta2_kernel = cp.ElementwiseKernel(
    'float32 a1, int32 y, float32 a2, int32 n',
    'float32 bt2',
    '''
    if (n < 0.1){
      bt2 = 0;
    }
    else{
      bt2 = lgamma(a1+y) + lgamma(a2+n-y) - lgamma(a1+a2+n) - lgamma(a1) - lgamma(a2) + lgamma(a1+a2);
    }
    ''',
    'beta2_kernel'
)


phiACC_kernel = cp.ElementwiseKernel(
    'float32 a1, float32 a2, int32 y, int32 n, float32 phi, float32 mu',
    'float32 temp_phi',
    '''
    if (n < 0.1 ){
      temp_phi = 0;
    }
    else{
      float temp_1 = 0.0 ;
      for (int i = 0; i < y; i++){
        temp_1 = temp_1 + 1 / (i + a1);
      }
      float temp_2 = 0.0 ;
      for (int i = 0; i < n-y; i++){
        temp_2 = temp_2 + 1 / (i + a2);
      }
      float temp_3 = 0.0 ;
      float phi_inv = 1 / phi;
      for (int i = 0; i < n; i++){
        temp_3 = temp_3 + 1 / (i + phi_inv);
      }
      temp_phi = - temp_1 * mu - temp_2 * (1 - mu) + temp_3;
    }
    ''',
    'phiACC_kernel'
)


bACC_kernel = cp.ElementwiseKernel(
    'float32 a1, float32 a2, int32 y, int32 n, float32 mu',
    'float32 temp_b',
    '''
    if ( n < 0.1 ){
      temp_b = 0;
    }
    else{
      float temp_1 = 0.0;
      for (int i = 0; i < y; i++){
        temp_1 = temp_1 + 1 / (i + a1);
      }
      float temp_2 = 0.0;
      for (int i = 0; i < n-y; i++){
        temp_2 = temp_2 + 1 / (i + a2);
      }
      temp_b = (temp_1 - temp_2) * mu * (1-mu);
    }
    ''',
    'bACC_kernel'
)

def cu_compute_randint_deriv_numpy(cu_linpred, cu_e_randint, cu_id_data, cu_e_y, cu_e_n, cu_e_phi, cu_inlcude_driv0 = False):
  lin = (cu_linpred + cp.matmul(cu_e_randint, cu_id_data)).astype(cp.float32)
  cu_e_mu = sigmoid2_kernel( lin )
  cu_e_mu = cp.clip(cu_e_mu,0.0000001, 0.9999999)

  cu_temp_1 = diff_digamma_kernel(cu_e_mu / cu_e_phi, cu_e_y) - diff_digamma_kernel((1 - cu_e_mu)/ cu_e_phi, cu_e_n - cu_e_y)

  cu_temp_2 = diff_trigamma_kernel(cu_e_mu / cu_e_phi, cu_e_y) + diff_trigamma_kernel((1 - cu_e_mu)/ cu_e_phi, cu_e_n - cu_e_y)

  cu_v1 = cu_temp_1 * cu_e_mu * (1 - cu_e_mu) / cu_e_phi
  cu_v2 = (cp.multiply(cp.multiply(cu_temp_2, cp.multiply(cu_e_mu,cu_e_mu)), cp.multiply(1- cu_e_mu, 1- cu_e_mu)) / (cu_e_phi * cu_e_phi) +
           cp.multiply(cp.multiply(cp.multiply(cu_temp_1, 1 - 2 * cu_e_mu), cu_e_mu), 1 - cu_e_mu) / cu_e_phi )
  cu_v1 = cp.matmul(cu_v1, cu_id_data.T)
  cu_v2 = cp.matmul(cu_v2, cu_id_data.T)

  if cu_inlcude_driv0:
    cu_v3 = beta2_kernel(cu_e_mu / cu_e_phi, cu_e_y, (1 - cu_e_mu)/ cu_e_phi, cu_e_n)
    cu_v3 = cp.matmul(cu_v3, cu_id_data.T)
    cu_v =  cp.concatenate((cu_v1,cu_v2,cu_v3),axis=0)
  else:
    cu_v =  cp.concatenate((cu_v1,cu_v2),axis=0)
  return cu_v

# Q-function for numpy
def cu_q_function(param, cu_x_q, cu_y_q, cu_n_q, cu_zmat_q, cu_ghq_weights_q, cu_randint_q, cu_randint_prec_q, cu_id_data_q,
                  cu_num_genes_q, cu_lower):
    q_start = time.time()
    cu_y_q = cu_y_q.astype(cp.int32)
    cu_n_q = cu_n_q.astype(cp.int32)
    #################################################################################################################################
    # 计算q-函数
    #数据升维
    cu_param = ((cp.asarray(param)).astype(cp.float32)).reshape(cu_num_genes_q,-1) # 我们输入的参数必须为一维的，这里我们把参数升级为原本的二维

    cu_b_q = cu_param[:,:-1]
    cu_b_q = cu_b_q.reshape(cu_b_q.shape[0],1,cu_b_q.shape[1]) # cu_b shape[5,1,2]
    cu_phi_q = cu_param[:,-1]
    cu_phi_q = cp.abs(cu_phi_q.reshape(cu_phi_q.shape[0],1,1)) # cu_phi_shape: [5,1,1] # 这里我们需要取 phi的绝对值
    cu_y_q = cu_y_q.reshape(cu_y_q.shape[0],cu_y_q.shape[1],1)
    cu_n_q = cu_n_q.reshape(cu_n_q.shape[0],cu_n_q.shape[1],1)


    cu_linpred_q = dot_product_kernel(cu_x_q, cu_b_q, axis = 2)
    cu_linpred_q = (cu_linpred_q).reshape(cu_num_genes_q,cu_zmat_q.shape[1],1)

    # print(f"cu_linpred_q:{cu_linpred_q.dtype}, cu_randint_q:{cu_randint_q.dtype}, cu_randint_prec_q:{cu_randint_prec_q.dtype}, cu_zmat_q:{cu_zmat_q.dtype}")
    cu_mu_q = sigmoid_kernel(cu_linpred_q, cu_randint_q, cu_randint_prec_q, cu_zmat_q)
    cu_mu_q = cp.clip(cu_mu_q,1e-5,1-1e-5)

    cu_alpha_1_q = cu_mu_q / cu_phi_q
    cu_alpha_2_q = (1 - cu_mu_q) / cu_phi_q

    cu_beta2 = beta2_kernel(cu_alpha_1_q, cu_y_q, cu_alpha_2_q, cu_n_q ) ## [5,3k,3] 并且在这里用deleter，消除误差
    cu_q_func_result = cp.sum(cu_beta2 ,axis=1) * cu_ghq_weights_q # # [5,3]

    # 最后把 五个基因的 q-fun 的结果加在一起
    cu_q_func_result = cu_lower - cp.sum(cu_q_func_result) # 这里仍然在GPU上

    #######################################################################################
    # 开始计算Q函数的导数
    # 求 b 的导数
    cu_deri_b_q1 = cu_x_q.reshape(cu_x_q.shape[0], cu_x_q.shape[1], cu_x_q.shape[2],1) # 升维成 [5, 3k, 2, 1]
    cu_deri_b_q2 = bACC_kernel(cu_alpha_1_q, cu_alpha_2_q, cu_y_q, cu_n_q, cu_mu_q)

    cu_deri_b_q2 = (cu_deri_b_q2).reshape(cu_deri_b_q2.shape[0], cu_deri_b_q2.shape[1], 1, cu_deri_b_q2.shape[2]) # dimension [5, 3k, 1, 3]
    cu_deri_b_q3 = cp.sum(cu_deri_b_q1 * cu_deri_b_q2, axis = 1) /cu_phi_q # dimension [5, 2, 3]
    cu_deri_b_q4 = cp.sum(cu_deri_b_q3 * cu_ghq_weights_q, axis = 2) # dimension [5,2]

    # phi 的导数
    cu_temp_phi_q = phiACC_kernel(cu_alpha_1_q, cu_alpha_2_q, cu_y_q, cu_n_q, cu_phi_q, cu_mu_q)

    cu_temp_phi_q = cp.sum(cu_temp_phi_q, axis=1) / np.square(cu_phi_q.reshape(-1,1)) # shape [5, 3]

    cu_deri_phi_q = cp.dot(cu_temp_phi_q, cu_ghq_weights_q.reshape(-1,1))

    # 合并导数，降维
    cu_deri = - cp.concatenate((cu_deri_b_q4, cu_deri_phi_q),axis=1).reshape(-1)
    cu_deri_q = cp.append(cu_deri, cu_q_func_result)

    cu_deri_q_result = (cp.asnumpy(cu_deri_q))
    cp.cuda.Stream.null.synchronize()
    end_q = time.time()
    return cu_deri_q_result[-1], cu_deri_q_result[:-1]

def cu_bbmix_lkl_agq_fixvar(cu_b_phi, cu_sigm2, cu_x, cu_randint, cu_id_data, cu_y, cu_n, cu_num_genes):
  cu_b = cu_b_phi[:, :-1]
  cu_b = cu_b.reshape(cu_b.shape[0], 1, cu_b.shape[1]) # shape[num_genes,1,2]
  cu_phi = cu_b_phi[:,-1]
  cu_phi = cu_phi.reshape(-1,1)
  cu_randint = cp.zeros_like(cu_randint)
  cu_linpred = dot_product_kernel(cu_x, cu_b, axis = 2)

  for i in range(3):
    cu_df = cu_compute_randint_deriv_numpy(cu_linpred, cu_randint, cu_id_data, cu_y, cu_n, cu_phi)
    cu_randint = cu_randint - 0.9 * (cu_df[:cu_num_genes,:] - cu_randint / cu_sigm2) / (cu_df[cu_num_genes:,:] - 1 / cu_sigm2)
  cp.cuda.Stream.null.synchronize()
  end_llkl_1 = time.time()


  cu_df = cu_compute_randint_deriv_numpy(cu_linpred, cu_randint, cu_id_data, cu_y, cu_n, cu_phi, cu_inlcude_driv0=True)
  cu_randint_prec = cp.abs(cu_df[cu_num_genes:cu_num_genes*2,:] - 1 / cu_sigm2)
  cu_fval = cu_df[cu_num_genes*2:,:] - cu_randint * cu_randint / cu_sigm2 / 2

  llkl = cp.sum(cu_fval - 0.5 * cp.log(cu_randint_prec),axis=1) - (int(cu_randint.shape[1]) * cp.log(cu_sigm2)/2).reshape(-1)
  return llkl


def cu_vem(cu_b_phi, cu_sigm2, cu_x, cu_randint, cu_randint_prec, cu_id_data, cu_y, cu_n, cu_ghq_weights, cu_ghq_nodes, cu_n_lap, cu_num_genes, cu_lower,
           cu_iteration, null):
  # E-M Algorithm
  start_time = time.time()

  # E-step
  start_e = time.time()
  cu_b = cu_b_phi[:, :-1]
  cu_b = cu_b.reshape(cu_b.shape[0], 1, cu_b.shape[1]) # shape[num_genes,1,2]
  cu_phi = cu_b_phi[:,-1]
  cu_phi = cu_phi.reshape(-1,1)
  cu_linpred = dot_product_kernel(cu_x, cu_b, axis = 2)

  for i in range(2):
    cu_df = cu_compute_randint_deriv_numpy(cu_linpred, cu_randint, cu_id_data, cu_y, cu_n, cu_phi)
    cu_randint = cu_randint - 0.9 * (cu_df[:cu_num_genes,:] - cu_randint / cu_sigm2) / (cu_df[cu_num_genes:,:] - 1 / cu_sigm2)
  cu_df = cu_compute_randint_deriv_numpy(cu_linpred, cu_randint, cu_id_data, cu_y, cu_n, cu_phi)
  cu_randint_prec = cp.abs(cu_df[cu_num_genes:,:] - 1 / cu_sigm2)
  cp.cuda.Stream.null.synchronize()
  end_e = time.time()
  #print("E-step used Time: " + str(end_e - start_e) + " s")

  # M - step
  start_m = time.time()
  cu_zmat = cp.multiply(cp.zeros((cu_y.shape[0],cu_y.shape[1],3)) + 1, cu_ghq_nodes).astype(cp.float32)
  cu_sigm2 =  (cp.mean( cu_randint * cu_randint + cp.reciprocal(cu_randint_prec), axis = 1)).reshape(-1,1)

  cu_b_phi = cp.asnumpy(cu_b_phi).astype(np.float32)
  if null == False:
    low_bound = (np.ones_like(cu_b_phi) * np.array([-np.inf, -np.inf, 1e-6]).reshape(1,-1)).reshape(-1) # 这里我们需要用有边界限制的优化器，这是下界
    up_bound = (np.ones_like(cu_b_phi) * np.array([np.inf, np.inf, np.inf]).reshape(1,-1)).reshape(-1) # 这里我们需要用有边界限制的优化器，这是上界
  else:
    low_bound = (np.ones_like(cu_b_phi) * np.array([-np.inf, 1e-6]).reshape(1,-1)).reshape(-1) # 这里我们需要用有边界限制的优化器，这是下界
    up_bound = (np.ones_like(cu_b_phi) * np.array([np.inf, np.inf]).reshape(1,-1)).reshape(-1) # 这里我们需要用有边界限制的优化器，这是上界

  cu_randint_vector = (cp.matmul(cu_randint, cu_id_data)).reshape(cu_num_genes, cu_zmat.shape[1], 1) # 提前计算a_i，并且变成和n一样大小的向量
  cu_precision_vector = (cp.matmul(cp.sqrt(cu_randint_prec), cu_id_data)).reshape(cu_num_genes, cu_zmat.shape[1], 1) # 提前计算precision，并且变成和n一样大小的向量

  #转化数据类型
  cu_randint_vector = cu_randint_vector.astype(cp.float32)
  cu_precision_vector = cu_precision_vector.astype(cp.float32)
  # 开始 优化函数
  cu_optim_result = minimize(cu_q_function, cu_b_phi.reshape(-1), method='L-BFGS-B', bounds = Bounds(low_bound,up_bound),
                            jac = True,
                            options={'maxls':100, 'ftol':1e7*np.finfo(float).eps},
                            args = (cu_x, cu_y, cu_n, cu_zmat, cu_ghq_weights, cu_randint_vector, cu_precision_vector, cu_id_data,
                            cu_num_genes, cu_lower))

  cu_b_phi = cp.asarray((cu_optim_result.x).reshape(cu_num_genes,-1))
  cu_b_phi = cu_b_phi.astype(cp.float32)
  cp.cuda.Stream.null.synchronize()
  end_m = time.time()
  #print("M-step used Time: " + str(end_m - start_m) + " s")
  #print(cu_optim_result)

  llkl = cu_bbmix_lkl_agq_fixvar(cu_b_phi, cu_sigm2, cu_x, cu_randint, cu_id_data, cu_y, cu_n, cu_num_genes)
  llkl = cp.asnumpy(llkl)
  cp.cuda.Stream.null.synchronize()
  end_llkl = time.time()
  #print("LLKL time: {} s".format(1*(end_llkl - end_m)))

  return cu_b_phi, cu_sigm2, llkl, cu_randint, cu_randint_prec


def daesc_cui(gene_index, num_iteration,
              ynxid_data,cu_id_data, cu_n_lap,initial_param, null, cum_time):
  # we wil block all codes, build a model which could train multiple genes at one time
  # in this function, we need to allow each genes getting different and suitable number of iteration of EM algorithm
  # 第一个很重要的参数 gene_index: 目前需要用DAESC模型拟合的基因的index

  # prepare some global data
  total_genes = int((ynxid_data.shape[0] - 1) / 2)
  cu_ghq_weights = cp.array([[0.1666667, 0.6666667, 0.1666667]]).astype(cp.float32).reshape(1,3)
  cu_ghq_nodes = cp.array([[-1.732051, -2.045201e-16, 1.732051]]).astype(cp.float32).reshape(1,3)

  # 初始参数 和 数据
  initial_param = initial_param[gene_index,:]
  if null == False: # under H1
    cu_param_result = cp.asarray(initial_param[:,[0,1,3]]).astype(cp.float32)
    cu_sigm2_result = cp.asarray(initial_param[:,[2]]).astype(cp.float32)
  else: # under H0
    cu_param_result = cp.asarray(initial_param[:,[4,7]]).astype(cp.float32)
    cu_sigm2_result = cp.asarray(initial_param[:,[6]]).astype(cp.float32)

  cu_randint_result = cp.zeros((len(gene_index),cu_id_data.shape[0]), dtype=cp.float32)
  cu_randint_prec_result = cp.ones_like(cu_randint_result,dtype = cp.float32) * 0

  cu_y_initial = (cp.asarray(ynxid_data[gene_index,:])).astype(cp.int32)
  cu_n_initial = (cp.asarray(ynxid_data[ np.array(gene_index) + total_genes,:])).astype(cp.int32)
  # cu_x_initial = cp.zeros((1,cu_y_initial.shape[1],2)).astype(cp.float32)
  cu_id_data = (cu_id_data).astype(cp.int32)

  if null == False:
    cu_x_initial = cp.zeros((1,cu_y_initial.shape[1],2))
    cu_x_initial[:,:,0] = cp.zeros((1,cu_y_initial.shape[1])) + 1
    cu_x_initial[:,:,1] = cp.asarray(ynxid_data[-1,:])
  else:
    cu_x_initial = cp.zeros((1,cu_y_initial.shape[1],1))
    cu_x_initial[:,:,0] = cp.zeros((1,cu_y_initial.shape[1])) + 1
  cu_x_initial = cu_x_initial.astype(cp.float32)

  gene_index = [i for i in range(len(gene_index))] # 这里，我们需要重新调整gene_index
  llkl_record_dict = {}
  speed_total = []
  for ele in gene_index:
    llkl_record_dict[ele] = [0]

  used_time = []

  # 开始 EM 算法
  for i in range(num_iteration):
    if len(gene_index) > 0:
      print()
      print("Iteration " + str(i) + " start: " + str(len(gene_index)) + " genes")
      start_em = time.time()

      # 在gene_index已知时，我们把数据提出来，放进DAESC模型里面
      start1 = time.time()
      cu_y = cu_y_initial[gene_index,:]
      cu_n = cu_n_initial[gene_index,:]
      cu_x = cu_x_initial[gene_index,:]

      cu_param = cu_param_result[gene_index,:] # b0, b1, phi
      cu_sigm2 = cu_sigm2_result[gene_index,:]
      cu_randint = cu_randint_result[gene_index,:]
      cu_randint_prec = cu_randint_result[gene_index,:]

      # 创建 cu_lower, 把q-function的值限制在很小的范围（每个基因贡献值小于5）
      cu_lower = 0
      if i != 0:
        for gene in gene_index:
          single_gene_llkl = llkl_record_dict[gene]
          cu_lower = cu_lower + single_gene_llkl[-1]
      cu_lower = cp.asarray(cu_lower, dtype = cp.float32)

      # EM 算法单次迭代
      cu_param, cu_sigm2, llkl, cu_randint, cu_randint_prec  = cu_vem(cu_param, cu_sigm2,
                                        cu_x, cu_randint, cu_randint_prec, cu_id_data, cu_y, cu_n,
                                        cu_ghq_weights, cu_ghq_nodes, cu_n_lap, len(gene_index), cu_lower,
                                        i, null)

      #开始把结果记录
      new_gene_index = []
      for gene in gene_index:
        (llkl_record_dict[gene])[0] += 1
        if llkl[gene_index.index(gene)] > (llkl_record_dict[gene])[-1] - 0.001 or i < 1:
          cu_param_result[gene_index,:] = cu_param
          cu_sigm2_result[gene_index,:] = cu_sigm2
          cu_randint_result[gene_index,:] = cu_randint
          cu_randint_prec_result[gene_index,:] = cu_randint_prec
          (llkl_record_dict[gene]).append(llkl[gene_index.index(gene)])
        if i < 10 :
          new_gene_index.append(gene)
        elif ( abs(((llkl_record_dict[gene])[-1] - (llkl_record_dict[gene])[-2]) / (llkl_record_dict[gene])[-1]) > 1e-7
            and (llkl_record_dict[gene])[-1] - (llkl_record_dict[gene])[-2] > - 0.001):
          new_gene_index.append(gene)
      gene_index = new_gene_index
      end_em = time.time()
      cum_time += end_em - start_em
      print(f"Iteration {i}, cummulative time = {cum_time} s")

  return cu_param_result, cu_sigm2_result, llkl_record_dict, cum_time





In [5]:
ynx_data = np.load("Simulation_Data/gene400_ynxid_data.npy")
ynx_data = ynx_data.T

id_data = np.load("Simulation_Data/one_hot_donor.npy")
id_data = id_data.T
cu_id_data = cp.asarray(id_data)

initial_param = np.load("Simulation_Data/gene400_initial_data.npy")

# Traing smaller dataset by T4 GPU

In [8]:
# Select gene index
# gene_index = [87,88, 89]
gene_index = [i for i in range(400)]
cum_time = 0.0

# Under H1
print(f"H1 Begins:")
my_param_result, my_sigm2_result, my_llkl_record_dict, cum_time = daesc_cui(gene_index,  num_iteration = 100,
                                                                            ynxid_data = ynx_data,
                                                                            cu_id_data = cu_id_data,
                                                                            cu_n_lap = 2 ,
                                                                            initial_param = initial_param,
                                                                            null = False, cum_time = cum_time)
# Under H0
print("\n \n H0 begins: \n")
my_param_null, my_sigm2_null, my_llkl_null, used_time_null = daesc_cui(gene_index,  num_iteration = 100,
                                                                            ynxid_data = ynx_data,
                                                                            cu_id_data = cu_id_data,
                                                                            cu_n_lap = 2 ,
                                                                            initial_param = initial_param,
                                                                            null = True, cum_time = cum_time)

print(f"Finally, we did finish training all genes using {cum_time} s")

H1 Begins:

Iteration 0 start: 400 genes
Iteration 0, cummulative time = 3.3834965229034424 s

Iteration 1 start: 400 genes
Iteration 1, cummulative time = 5.301051139831543 s

Iteration 2 start: 400 genes
Iteration 2, cummulative time = 6.907317399978638 s

Iteration 3 start: 400 genes
Iteration 3, cummulative time = 8.518257141113281 s

Iteration 4 start: 400 genes
Iteration 4, cummulative time = 10.36640453338623 s

Iteration 5 start: 400 genes
Iteration 5, cummulative time = 12.652134418487549 s

Iteration 6 start: 400 genes
Iteration 6, cummulative time = 14.424187660217285 s

Iteration 7 start: 400 genes
Iteration 7, cummulative time = 15.901460647583008 s

Iteration 8 start: 400 genes
Iteration 8, cummulative time = 17.64350390434265 s

Iteration 9 start: 400 genes
Iteration 9, cummulative time = 19.47019100189209 s

Iteration 10 start: 400 genes
Iteration 10, cummulative time = 21.03881573677063 s

Iteration 11 start: 299 genes
Iteration 11, cummulative time = 22.40534567832946

In [9]:
# Calculate Type I error
# First, under H1
llkl_h1 = []
for key in my_llkl_record_dict.keys():
  llkl_h1.append(my_llkl_record_dict[key][-1])
llkl_h1 = np.array(llkl_h1)

# Second, under H0
llkl_null = []
for key in my_llkl_null.keys():
  llkl_null.append(my_llkl_null[key][-1])
llkl_null = np.array(llkl_null)

llkl2 = np.concatenate( (llkl_h1.reshape(-1,1), llkl_null.reshape(-1,1)), axis = 1)
tyep1 = np.mean(chi2.sf(2*(llkl2[:,0] - llkl2[:,1]), 1) < 0.05)
print(f"Type I error: {tyep1}")

Type I error: 0.0675


# Traing smaller dataset by A100 GPU

In [6]:
# Select gene index
# gene_index = [87,88, 89]
gene_index = [i for i in range(400)]
cum_time = 0.0

# Under H1
print(f"H1 Begins:")
my_param_result, my_sigm2_result, my_llkl_record_dict, cum_time = daesc_cui(gene_index,  num_iteration = 100,
                                                                            ynxid_data = ynx_data,
                                                                            cu_id_data = cu_id_data,
                                                                            cu_n_lap = 2 ,
                                                                            initial_param = initial_param,
                                                                            null = False, cum_time = cum_time)
# Under H0
print("\n \n H0 begins: \n")
my_param_null, my_sigm2_null, my_llkl_null, used_time_null = daesc_cui(gene_index,  num_iteration = 100,
                                                                            ynxid_data = ynx_data,
                                                                            cu_id_data = cu_id_data,
                                                                            cu_n_lap = 2 ,
                                                                            initial_param = initial_param,
                                                                            null = True, cum_time = cum_time)

print(f"Finally, we did finish training all genes using {cum_time} s")

H1 Begins:

Iteration 0 start: 400 genes
Iteration 0, cummulative time = 13.35936188697815 s

Iteration 1 start: 400 genes
Iteration 1, cummulative time = 14.150588035583496 s

Iteration 2 start: 400 genes
Iteration 2, cummulative time = 14.80172848701477 s

Iteration 3 start: 400 genes
Iteration 3, cummulative time = 15.385123491287231 s

Iteration 4 start: 400 genes
Iteration 4, cummulative time = 15.997272491455078 s

Iteration 5 start: 400 genes
Iteration 5, cummulative time = 16.602239847183228 s

Iteration 6 start: 400 genes
Iteration 6, cummulative time = 17.194197416305542 s

Iteration 7 start: 400 genes
Iteration 7, cummulative time = 17.82461643218994 s

Iteration 8 start: 400 genes
Iteration 8, cummulative time = 18.428810119628906 s

Iteration 9 start: 400 genes
Iteration 9, cummulative time = 19.035770654678345 s

Iteration 10 start: 400 genes
Iteration 10, cummulative time = 19.602322816848755 s

Iteration 11 start: 306 genes
Iteration 11, cummulative time = 20.0618643760

In [7]:
# Calculate Type I error
# First, under H1
llkl_h1 = []
for key in my_llkl_record_dict.keys():
  llkl_h1.append(my_llkl_record_dict[key][-1])
llkl_h1 = np.array(llkl_h1)

# Second, under H0
llkl_null = []
for key in my_llkl_null.keys():
  llkl_null.append(my_llkl_null[key][-1])
llkl_null = np.array(llkl_null)

llkl2 = np.concatenate( (llkl_h1.reshape(-1,1), llkl_null.reshape(-1,1)), axis = 1)
tyep1 = np.mean(chi2.sf(2*(llkl2[:,0] - llkl2[:,1]), 1) < 0.05)
print(f"Type I error: {tyep1}")

Type I error: 0.0725
